# A Very Brief Introduction to NWM Subsetting

This notebooks provides a very brief introduction to the process of subsetting NWM forecast results for small watershed areas using Thredds and OpenDAP technologies.  The goal of this notebook is to stimulate thinking about how subsetting can be standardized and applied to a larger portion of the NWM forcast archive. 

In [ ]:
import re
import xarray
import requests
import warnings
import matplotlib
import numpy as np
import holoviews as hv
from matplotlib import rc
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from matplotlib.animation import FuncAnimation


rc('animation', html='html5')
hv.extension('bokeh', 'matplotlib')
plt.rcParams["animation.html"] = "jshtml"
warnings.filterwarnings('ignore')

%matplotlib inline

Define the url of the Thredds server that's hosting the data we're interested in. CUAHSI and RENCI maintain a public archive of ~40Tb of data at [http://thredds.hydroshare.org/thredds](http://thredds.hydroshare.org/thredds).

In [ ]:
# define the url of the thredds server
thredds_base='http://thredds.hydroshare.org/thredds'

We need to specify which forcast and date we're interested in.  The easiest way to do this is by navigating to the link above, choosing valid parameters, and entering them in the cell below.

In [ ]:
# specify date and forcast arguments
date = '20170910'
configuration = 'medium_range'
output_type = 'land'

# build the full catalog url and define the namespace for XML parsing
catalog = '%s/catalog/nwm/harvey/nwm.%s/%s/catalog.xml' % (thredds_base, date, configuration)
namespaces = {'unidata': 'http://www.unidata.ucar.edu/namespaces/thredds/InvCatalog/v1.0'}

Read the Thredds catalog and parse the list of files that are available to us.

In [ ]:
r = requests.get(catalog)
tree = ET.ElementTree(ET.fromstring(r.text))
root = tree.getroot()

In [ ]:
# filter to our data of interest
unidata_datasets = root.findall('unidata:dataset/unidata:dataset', namespaces)
pattern = "^.*t00z.%s.%s.*$" % (configuration, output_type)
paths = [] 
for ds in unidata_datasets:
    name = ds.attrib['name']
    if re.match(pattern, name) is not None:
        paths.append('/'.join([thredds_base, 'dodsC', ds.attrib['urlPath']]))

## 1. Subset Gridded Data

Using Thredds and OpenDAP, we can quickly open and inspect large NetCDF files. To fully appreciate the strength of this approach, compare the timed output of the cell below with the time it takes to download the same file: http://thredds.hydroshare.org/thredds/catalog/nwm/harvey/nwm.20170910/medium_range/catalog.html

In [ ]:
%%time
single_dataset = xarray.open_dataset(paths[10])

In [ ]:
single_dataset

We can also open multiple datasets without downloading them.  A single medium range land forecast is approximately 20GB.

In [ ]:
%%time 
mds = xarray.open_mfdataset(paths)

To preview the data, simply inspect the object above using this following commands:

```
mds -> list the general metadata for the dataset
list(mds.variables) -> list all variables
list(mds.indexes) -> list indices
```

In [ ]:
mds

In [ ]:
list(mds.indexes)

In [ ]:
list(mds.variables)

Determine x, y indices for area of interest and subset this dataset to isolate only the area we're interested in.  In this case, we'll use the `120200` HUC6 that was affected by Hurricane Harvey.  The GIS data for this resource can be found here: https://www.hydroshare.org/resource/adb14c9c073e4eee8be82fadb21a0a93/.  I've already determined the bounding box for this watershed boundary, however this process could be streamline using GIS libraries.

In [ ]:
lon_max = -770000.0
lon_min = -1060000.0
lat_max = 218000.0
lat_min = -5000.0

Find the `X` and `Y` indices that are within the latitude and longitude bounds above.

In [ ]:
lats = mds.variables['y'][:] 
lons = mds.variables['x'][:]
lat_bnds = [ lat_min , lat_max ]
lon_bnds = [ lon_min , lon_max ] 

lat_inds = np.where((lats > lat_bnds[0]) & (lats < lat_bnds[1]))
lon_inds = np.where((lons > lon_bnds[0]) & (lons < lon_bnds[1]))

In [ ]:
llat = lat_inds[0].min()
ulat = lat_inds[0].max()
llon = lon_inds[0].min()
ulon = lon_inds[0].max()

print('latitude index range:  %d -> %d' % (llat, ulat))
print('longitude index range: %d -> %d' % (llon, ulon))

Using the indices above, all variables of the data can be subsetted in a single command.

In [ ]:
# perform subset
subset = mds.isel(x=lon_inds[0], y=lat_inds[0])

# view the subsetted data
subset

In [ ]:
# preview the SOILSAT variable
subset.SOILSAT_TOP[0][:][:].values

Quickly preview the data using this xarray plot function

In [ ]:
subset.SOILSAT_TOP[0].plot()

It's more interesting to look at an animation though :) 

In [ ]:
%%capture

sat = subset.SOILSAT_TOP[range(0,80,10)]

fig = plt.figure()
ax1 = fig.add_subplot(1,1,1)

def animate(i):
    data = sat[i]
    ax1.pcolormesh(data)

anim = FuncAnimation(fig,animate,frames=8)

In [ ]:
# render the animation.  This may take several minutes.
anim

Save the subsetted `land` forecasts

In [ ]:
sat.to_netcdf('120200_land_sat.nc')

## 2. Subset Channel Data 

Perform similar steps as those outlined above (gridded data) for the reach level forecasts. One important difference is that these data are index by unique COMID identifiers.  As a result these data need to be spatially intersected with an area of interest prior to subsetting them.  First, however define the configuration and output type `channel_rt` that we want to read:

In [ ]:
# specify date and forecast arguments
configuration = 'medium_range'
output_type = 'channel_rt'

In [ ]:
# filter to our data of interest
unidata_datasets = root.findall('unidata:dataset/unidata:dataset', namespaces)
pattern = "^.*t00z.%s.%s.*$" % (configuration, output_type)
paths = [] 
for ds in unidata_datasets:
    name = ds.attrib['name']
    if re.match(pattern, name) is not None:
        paths.append('/'.join([thredds_base, 'dodsC', ds.attrib['urlPath']]))

Similar to the process for gridded data, read all Thredds data paths into memory using the `open_mfdataset` function. This will take 30-60 seconds to complete.

In [ ]:
%%time 
channel_ds = xarray.open_mfdataset(paths)

Quickly preview the data to see the variables and indices

In [ ]:
channel_ds

For this subsetting exercise we'll use a list of COMIDs that exist in the HUC6 watershed that was used in the first portion of the notebook. However, this process can be generalized using ArcPy or GDAL python bindings.

In [ ]:
reachids = [1114115, 1114117, 1114119, 1114129, 1114133, 1114135, 1114143, 1114145, 1114147, 1114149, 1114151, 1114153, 1114155, 1114157, 1114159, 1114161, 1114163, 1114165, 1114167, 1114169, 1114171, 1114173, 1114175, 1114177, 1114179, 1114181, 1114183, 1114185, 1114187, 1114189, 1114191, 1114193, 1114195, 1114197, 1114199, 1114201, 1114203, 1114205, 1114207, 1114209, 1114211, 101, 1114215, 1114217, 1114219, 1114221, 1114223, 1114225, 1114227, 1114229, 1114231, 1114233, 1114235, 1114237, 1114239, 1114241, 1114243, 1114245, 1114247, 1114249, 1114251, 1114253, 1114255, 1114257, 1114259, 1114261, 1114263, 1114265, 1114267, 1114269, 1114271, 1114273, 1114275, 1114277, 1114279, 1114281, 1114283, 1114285, 1114287, 1114289, 1114291, 1114293, 1114295, 1114297, 1114299, 1114301, 1114303, 1114305, 1114307, 1114309, 1114311, 1114313, 1114315, 1114317, 1114319, 1114321, 1114323, 1114325, 1114327, 1114329, 1114331, 1114333, 1114335, 1114337, 1114339, 4456676, 1114341, 1114343, 4456680, 1114345, 4456682, 1114347, 1114349, 1114353, 1114355, 1114357, 1114359, 1114361, 1114363, 1114365, 1114367, 1114369, 1114371, 1114373, 1114375, 1114377, 1114379, 1114381, 1114383, 1114385, 1114387, 1114389, 1114391, 1114393, 1114395, 1147165, 1114399, 1114401, 1114403, 1114405, 1114407, 1114409, 1114411, 1114413, 1114415, 1114417, 1114419, 1114421, 1114423, 1114425, 1114427, 1114429, 1114431, 1114433, 1114435, 1114437, 1114439, 1114441, 1147211, 1114445, 1114447, 1114449, 1147219, 1147221, 1147223, 1147225, 1147227, 1147229, 1147231, 1147233, 1147235, 1147237, 1147239, 1147241, 1147243, 1147245, 1147247, 1147249, 1147251, 1147253, 1147255, 1147257, 1147259, 1147261, 1147263, 1147265, 1147267, 1147269, 1147271, 1147273, 1147275, 1147277, 1147279, 1147281, 1147283, 1147285, 1147287, 1147289, 1147291, 1147293, 1147295, 1147297, 1147299, 1147377, 1147379, 1147381, 1147383, 1147385, 1147387, 1147389, 1147391, 1147393, 1147395, 1147399, 1147401, 1147403, 1147405, 1147407, 1147409, 1147411, 1147413, 1114213, 1475243, 1475667, 1148213, 1148215, 1148217, 1148219, 1148221, 1148223, 1148225, 1148227, 1148229, 1148231, 1148233, 1148235, 1148237, 1148239, 1148241, 1148243, 1148245, 1148247, 1148249, 1148251, 1148253, 1148255, 1148257, 1148259, 1148261, 1148263, 1148265, 1148267, 1148269, 1148271, 1148273, 1148275, 1148277, 1148279, 1148281, 1148283, 1148285, 1148287, 1148289, 1148291, 1148293, 1148295, 1115529, 1148299, 1148301, 1148303, 1148305, 1148307, 1148309, 1148311, 1148313, 1115547, 1148317, 1148319, 1148321, 1148323, 1148325, 1148327, 1148329, 1148331, 1148333, 1148335, 1148337, 1148339, 1148341, 1148343, 1148345, 1148347, 1148349, 1148351, 1148353, 1148355, 1148357, 1148359, 1148361, 1148363, 1148365, 1148367, 1148369, 1148371, 1148373, 1148375, 1148377, 1148379, 1148381, 1148383, 1148385, 1148387, 1148389, 1148391, 1148393, 1148395, 1148397, 1148399, 1148401, 1148443, 1148445, 1148447, 1148449, 1148451, 1148453, 1148455, 1148457, 1148461, 1148463, 1148465, 1148467, 1148469, 1148471, 1148473, 1148475, 1148477, 1148479, 1148481, 1148483, 1148485, 1148487, 1148489, 1148491, 1148493, 1148495, 1148497, 1148499, 1148501, 1148503, 1148505, 1148507, 1148509, 1148511, 1148513, 1148515, 1148517, 1148519, 1148521, 1148523, 1148525, 1115759, 1115761, 1148531, 1148533, 1148535, 1148537, 1148539, 1148541, 1115797, 1115799, 1115801, 1115803, 1115805, 1115807, 1115809, 1115811, 1115813, 1115815, 1115819, 1115821, 1115825, 1147167, 1147169, 1147171, 1147173, 1147175, 1147177, 1147179, 1147181, 1147183, 1147185, 1147187, 1147189, 1147191, 1147193, 1147195, 1147197, 1147199, 1147201, 1147203, 1147205, 1147207, 1147209, 1147213, 1147215, 1147217, 1149703, 1149705, 1149707, 1149709, 1149711, 1149713, 1149715, 1149717, 1149719, 1149721, 1149723, 1149725, 1149727, 1149729, 1149731, 1149733, 1149735, 1149737, 1149739, 1149741, 1149743, 1149745, 1149747, 1149749, 1149751, 1149753, 1149755, 1149757, 1149759, 1149761, 1149763, 1149765, 1149767, 1149769, 1149771, 1149773, 1149775, 1149777, 1149779, 1149783, 1149785, 1149787, 1149789, 1149791, 1149793, 1149795, 1149797, 1149799, 1149801, 1149803, 1149805, 1149807, 1149809, 1149811, 1149813, 1149815, 1149817, 1149819, 1149821, 1149823, 1149825, 1149827, 1149829, 1149831, 1149833, 1149835, 1149837, 1149839, 1149841, 1149843, 1149845, 1149847, 1149849, 1149851, 1149853, 1149855, 1149857, 1149859, 1149861, 1149863, 1149865, 1149867, 1149869, 1149871, 1149873, 1149875, 1149877, 1149879, 1149881, 1149883, 1149885, 1149887, 1149889, 1149891, 1149893, 1149895, 1149897, 1149899, 1149901, 1149903, 1149905, 1149907, 1149909, 1149911, 1149913, 1149915, 1149917, 1149919, 1149921, 1149923, 1149925, 1149927, 1149929, 1149931, 1149933, 1149935, 1149937, 1149939, 1149941, 1149943, 1149945, 1149947, 1149949, 1149951, 1149953, 1149955, 1149957, 1149959, 1149961, 1149963, 1149965, 1149967, 1149969, 1149971, 1149973, 1149975, 1149977, 1149979, 1149981, 1149983, 1149985, 1149989, 1149991, 1149993, 1149995, 1149997, 1149999, 1150001, 1150003, 1150005, 1150007, 1150009, 1150011, 1150013, 1150015, 1477697, 1150019, 1150021, 1150023, 1150025, 1150027, 1150029, 1150031, 1150033, 1150035, 1150037, 1150039, 1150041, 1150043, 1150045, 1150047, 1150049, 1150051, 1150053, 1150055, 1150057, 1150059, 1150061, 1150063, 1150065, 1150067, 1150069, 1150071, 1150073, 1150075, 1150077, 1150079, 1150081, 1150083, 1150085, 1150087, 1150089, 1150091, 1150093, 1150095, 1150097, 1150099, 1150101, 1150103, 1150105, 1150107, 1150109, 1150111, 1150113, 1150115, 1150117, 1150119, 1150121, 1150123, 1150125, 1150127, 1150129, 1150131, 1150133, 1150135, 1150137, 1150139, 1150141, 1150143, 1150145, 1150147, 1150149, 1150151, 1150153, 1150155, 1150157, 1150159, 1150161, 1150163, 1150165, 1150167, 1150169, 1150171, 1150173, 1150175, 1150177, 1150179, 1150181, 1150183, 1150185, 1150187, 1150189, 1150191, 1150193, 1150195, 1150197, 1150199, 1150201, 1150203, 1150205, 1150207, 1150209, 1150211, 1150213, 1150215, 1150217, 1150219, 1150221, 1150223, 1150225, 1150227, 1150229, 1150231, 1150233, 1150235, 1150237, 1150239, 1150241, 1150243, 1150245, 1150247, 1150249, 1150251, 1150253, 1150255, 1150257, 1150259, 1150261, 1150263, 1150265, 1150267, 1150269, 1150271, 1150273, 1150275, 1150277, 1150279, 1150281, 1150283, 1150285, 1150287, 1150289, 1150291, 1150293, 1150295, 1150297, 1150299, 1150301, 1150303, 1150305, 1150307, 1150309, 1150311, 1150313, 1150315, 1150317, 1150319, 1150321, 1150323, 1150325, 1150327, 1150329, 1150331, 1150333, 1150337, 1150339, 1150341, 1150447, 1150449, 1150451, 1150453, 1150455, 1150457, 1150459, 1150461, 1150463, 1150465, 1150467, 1150469, 1150471, 1150473, 1150475, 1150477, 1150479, 1150481, 1150483, 1150485, 1150487, 1150489, 1150491, 1150493, 1150495, 1150497, 1150499, 1150501, 1150503, 1150505, 1150507, 1150509, 1150511, 1150513, 1150515, 1150517, 1150519, 1150521, 1150523, 1150525, 1150527, 1150529, 1150531, 1150533, 1150535, 1150537, 1150539, 1150541, 1150543, 1150545, 1150547, 1150549, 1150551, 1150553, 1150555, 1150557, 1150559, 1150561, 1150563, 1150565, 1150567, 1150569, 1150571, 1150573, 1150575, 1150577, 1150579, 1150581, 1150583, 1150585, 1150587, 1150589, 1150591, 1150593, 1150595, 1150597, 1150599, 1150601, 1150603, 1150605, 1150607, 1150609, 1150611, 1150613, 1150615, 1150617, 1151355, 1151357, 1151359, 1151361, 1151363, 1151365, 1151367, 1151369, 1151371, 1151373, 1151375, 1151377, 1151379, 1151381, 1151383, 1151385, 1151387, 1151389, 1151391, 1151393, 1151395, 1118629, 1118631, 1118633, 1118635, 1118637, 1151407, 1151409, 1151411, 1151413, 1151415, 1151417, 1151419, 1151421, 1151423, 1151425, 1151427, 1151429, 1151431, 1151433, 1151435, 1151437, 1151439, 1151441, 1151443, 1151445, 1151447, 1151449, 1151451, 1151453, 1151455, 1151457, 1151459, 1151461, 1151463, 1151465, 1151467, 1151469, 1151471, 1151473, 1151475, 1151477, 1151479, 1151483, 1151485, 1151487, 1151489, 1151491, 1151493, 1151495, 1151497, 1151499, 1151501, 1151503, 1151505, 1151507, 1151509, 1151511, 1151513, 1151515, 1151517, 1151519, 1151521, 1151523, 1151525, 1151527, 1151529, 1151531, 1151533, 1151535, 1151537, 1151539, 1151541, 1151543, 1151545, 1151547, 1151549, 1151551, 1151553, 1151555, 1151557, 1151559, 1151561, 1151563, 1151565, 1151567, 1151569, 1151571, 1151573, 1151575, 1151577, 1151579, 1151581, 1151583, 1151585, 1151587, 1151589, 1151591, 1151593, 1151595, 1151597, 1151599, 1151601, 1151603, 1151605, 1151607, 1151609, 1151611, 1151613, 1151615, 1151617, 1151619, 1151621, 1151623, 1151625, 1151627, 1151629, 1151631, 1151633, 1151635, 1151637, 1151639, 1151641, 1151643, 1151647, 1151649, 1151651, 1151653, 1151655, 1151657, 1151659, 1151661, 1151663, 1151665, 1151667, 1151669, 1151671, 1151673, 1151675, 1151677, 1151679, 1151681, 1151683, 1151685, 1151687, 1151689, 1151691, 1151693, 1151695, 1151697, 1151699, 1151701, 1151703, 1151705, 1151707, 1151709, 1151711, 1151713, 1151715, 1151717, 1151719, 1151721, 1151723, 1151725, 1151727, 1151729, 1151731, 1151733, 1151735, 1151737, 1151739, 1151741, 1151743, 1151745, 1151747, 1151749, 1151751, 1151753, 1151755, 1151757, 1151759, 1151761, 1151763, 1151765, 1151767, 1151769, 1151771, 1151773, 1151777, 1151779, 1151781, 1151783, 1151785, 1151787, 1151789, 1151791, 1151793, 1151795, 1151797, 1151799, 1151801, 1151803, 1151805, 1151807, 1151809, 1151811, 1151929, 1151931, 1151933, 1151935, 1151937, 1151939, 1151941, 1151943, 1151945, 1151947, 1151949, 1151951, 1151953, 1151955, 1151957, 1151959, 1151961, 1151963, 1151965, 1151967, 1151969, 1151971, 1151973, 1151975, 1151977, 1151979, 1152229, 1152231, 1152233, 1152235, 1152237, 1152239, 1152241, 1152243, 1152247, 1152249, 1152251, 1152253, 1152255, 1152257, 1152259, 1152261, 1152263, 1152265, 1152267, 1152269, 1152271, 1152273, 1152275, 1152277, 1152279, 1152281, 1152283, 1152285, 1152287, 1152341, 1152343, 1152345, 1152347, 1152349, 1152351, 1152353, 1152355, 1152357, 1152359, 1153777, 1153779, 4452268, 4452270, 4452272, 1142706, 4452276, 1142710, 4452280, 4452282, 1142716, 1142718, 1142720, 1142722, 4452292, 4452294, 4452296, 4452298, 4452300, 4452302, 4452304, 4452306, 4452308, 4452310, 4452312, 4452314, 4452316, 4452318, 4452320, 4452322, 1142756, 4452326, 4452328, 4452330, 4452332, 1142766, 4452336, 4452340, 4452342, 4452344, 4452346, 4452348, 1148297, 1115531, 1148315, 8331684, 1115757, 1148527, 1148529, 1115769, 1115773, 1143098, 1143100, 1143102, 1143114, 1143128, 4452708, 1157297, 1157299, 1157301, 1157303, 1157305, 1157307, 1157309, 1157311, 1157313, 1157315, 1157317, 1157319, 1157321, 1157323, 1157325, 1157327, 1157329, 1157331, 1157333, 1157335, 1157337, 1157339, 1157341, 1157343, 1157345, 1157347, 1157349, 1157351, 1157353, 1157355, 1157357, 1157359, 1157361, 1157363, 1157365, 1157367, 1157369, 1157371, 1157373, 1157375, 1157377, 1157379, 1157381, 1157383, 1157385, 1157387, 1157389, 1157391, 1157393, 1157395, 1157397, 1157399, 1157401, 1157403, 1157405, 1157407, 1157409, 1157411, 1157413, 1157415, 1157417, 1157419, 1157421, 1157423, 1157425, 1157427, 1157429, 1157431, 1157433, 1157435, 1157437, 1157439, 1157441, 1157443, 1157445, 1157447, 1157449, 1157451, 1157453, 1157455, 1157457, 1157459, 1157461, 1157463, 1157465, 1157467, 1157469, 1157471, 1157473, 1157475, 1157477, 1157479, 1157481, 1157483, 1157485, 1157487, 1157489, 1157491, 1157493, 1157495, 1157497, 1157499, 1157501, 1157503, 1157505, 1157507, 1157509, 1157511, 1157513, 1157515, 1157517, 1157519, 1157521, 1157523, 1157525, 1157527, 1157529, 1157531, 1157533, 1157535, 1157537, 1157539, 1157541, 1157543, 1157545, 1157547, 1157549, 1157551, 1157553, 1157555, 1157557, 1157559, 1157561, 1157563, 1157565, 1157567, 1157569, 1157571, 1157573, 1157575, 1157577, 1157579, 1157581, 1157583, 1157585, 1157587, 1157589, 1157591, 1157593, 1157595, 1157597, 1157599, 1157601, 1157603, 1157605, 1157607, 1157609, 1157611, 1157613, 1157615, 1157617, 1157619, 1157621, 1157623, 1157625, 1157627, 1157629, 1157631, 1157633, 1157635, 1157637, 1157639, 1157641, 1157643, 1157645, 1157647, 1157649, 1157651, 1157653, 1157655, 1157657, 1157659, 1157661, 1157663, 1157665, 1157667, 1157669, 1157673, 1157675, 1157677, 1157679, 1157681, 1157683, 1157685, 1157687, 1157689, 1157691, 1157693, 1157695, 1157697, 1157699, 1157701, 1157703, 1157705, 1157707, 1157709, 1157711, 1157713, 1157715, 1157717, 1157719, 1157721, 1157723, 1157725, 1157727, 1157729, 1157733, 1157735, 1157737, 1157739, 1157741, 1157743, 1157745, 1157747, 1157749, 1157751, 1157753, 1157755, 1157757, 1157759, 1157761, 1157763, 1157765, 1157767, 1157769, 1157771, 1157773, 1157775, 1157777, 1157779, 1157781, 1157783, 1157785, 1157787, 1157789, 1157791, 1157793, 1157795, 1157797, 1157799, 1157801, 1157803, 1157805, 1157807, 1157809, 1157811, 1157813, 1157815, 1157819, 1157821, 1157825, 1157827, 1143244, 1143246, 1143248, 1143250, 1143252, 1143254, 1143256, 4452826, 4452828, 1157933, 1157935, 1157937, 1157939, 1157941, 1157943, 1157945, 1157947, 1157949, 1157951, 1157953, 1157955, 1157957, 1157959, 1157961, 1157963, 1157965, 1157967, 1157969, 1157971, 1157973, 1157975, 1157977, 1157979, 1157981, 1157983, 1157985, 1157987, 1157989, 1157991, 1157993, 1157995, 1157997, 1157999, 1158001, 1158003, 1158005, 1158007, 1158009, 1158011, 1158013, 1158015, 1158017, 1158019, 1158021, 1158023, 1158025, 1158027, 1158029, 1158031, 1158033, 1158035, 1158037, 1158039, 1158041, 1158043, 1158045, 1158047, 1158049, 1158051, 1158053, 1158055, 1158057, 1158059, 1158061, 1158063, 1158065, 1158067, 1158069, 1158071, 1158073, 1158075, 1158077, 1158079, 1158081, 1158083, 1158085, 1158087, 1158089, 1158091, 1158093, 1158095, 1158097, 1158099, 1158101, 1158103, 1158105, 1158107, 1158109, 1158111, 1158113, 1158115, 1158117, 1158119, 1158121, 1158123, 1143294, 4452864, 4452866, 4452868, 4452870, 4452872, 4452874, 1143312, 4452882, 4452884, 1147163, 1158469, 1158471, 1158473, 1158475, 1158477, 1158479, 1158481, 1158483, 1158485, 1158487, 1158489, 1158491, 1158493, 1158495, 1158497, 1158499, 1158501, 1158503, 1158505, 1158507, 1158509, 1158511, 1158513, 1158515, 1158517, 1158519, 1158521, 1158523, 1158525, 1158527, 1158529, 1158531, 1158533, 1158535, 1158537, 1158539, 1158541, 1158543, 1158589, 1158591, 1158593, 1158595, 1158597, 1158599, 1158601, 1158603, 1158605, 1158607, 1158609, 1158611, 1158613, 1158615, 1158617, 1158861, 1158863, 1158865, 1158867, 1158869, 1158871, 1158873, 1158875, 1158877, 1158879, 1158881, 1158883, 1158885, 1158887, 1158889, 1158891, 1158893, 1158895, 1158897, 1158899, 1158901, 1158903, 1158905, 1158907, 1158909, 1158911, 1158913, 1158915, 1158917, 1158919, 1158921, 1158923, 1158925, 1158927, 1158929, 1158931, 1158933, 1158935, 1158937, 1158939, 1158941, 1158943, 1158945, 1158947, 1158949, 1158951, 1158953, 1158955, 1158957, 1158959, 1158961, 1158963, 1158965, 1158967, 1158969, 1158971, 1158973, 1158975, 1158977, 1158979, 1158981, 1158983, 1158985, 1158987, 1158989, 1158991, 1158993, 1158995, 1158997, 1158999, 1159001, 1159003, 1159007, 1159009, 1159011, 1159013, 1159015, 1159017, 1159019, 1159021, 1159023, 1159025, 1159027, 1159029, 1159031, 1159033, 1159035, 1159037, 1159039, 1159041, 1159043, 1159045, 1159047, 1159049, 1159051, 1159053, 1159055, 1159057, 1159059, 1159061, 1159063, 1159065, 1159067, 1159069, 1159071, 1159073, 1159075, 1159077, 1159079, 1159081, 1159083, 1159085, 1159087, 1159089, 1159091, 1159093, 1159095, 1159097, 1159099, 1159101, 1159103, 1159105, 1159107, 1159109, 1159111, 1159113, 1159115, 1159117, 1159119, 1159121, 1159123, 1159125, 1159127, 1159129, 1159131, 1159133, 1159135, 1159137, 1159139, 1159141, 1159143, 1159145, 1159147, 1159149, 1159151, 1159153, 1159155, 1159157, 1159159, 1159161, 1159163, 1159165, 1159167, 1159169, 1159171, 1159173, 1159175, 1159177, 1159179, 1159181, 1159183, 1159185, 1159187, 1159189, 1159191, 1159193, 1159195, 1159197, 1159199, 1159201, 1159203, 1159205, 1159211, 1159213, 1159215, 1159217, 1159219, 1159221, 1159223, 1159225, 1159227, 1159229, 1159231, 1159233, 1159235, 1159237, 1159239, 1159241, 1159243, 1159245, 1159247, 1159249, 1159251, 1159253, 1159303, 1159305, 1159307, 1159309, 1159311, 1159313, 1159317, 1159325, 1159327, 1159329, 1159331, 1159335, 1159337, 1159339, 1159343, 1159345, 1159347, 1159349, 1159351, 1159353, 1159355, 1159357, 1159359, 1159361, 1159363, 1159365, 1159367, 1159369, 1159371, 1159373, 1159375, 1159377, 1159379, 1159381, 1159383, 1159385, 1159387, 1159389, 1159391, 1159393, 1159395, 1159397, 1159399, 1159401, 1159403, 1159405, 1159407, 1159409, 1159411, 1159413, 1159415, 1159417, 1159419, 1159421, 1159423, 1159425, 1159427, 1159429, 1159431, 1159433, 1159435, 1159437, 1159439, 1159441, 1159443, 1159445, 1159447, 1159449, 1159451, 1159453, 1159455, 1159457, 1159459, 1159461, 1159463, 1159465, 1159467, 1159469, 1159471, 1159473, 1159475, 1159477, 1159479, 1159481, 1159483, 1159485, 1159487, 1159489, 1159491, 1159493, 1159495, 1159497, 1159499, 1159501, 1159503, 1159505, 1159507, 1159509, 1159511, 1159513, 1159515, 1159517, 1159519, 1159521, 1159523, 1159525, 1159527, 1159529, 1159531, 1159533, 1159535, 1159537, 1159539, 1159541, 1159543, 1159545, 1159547, 1159549, 1159551, 1159553, 1159555, 1159557, 1159559, 1159561, 1159563, 1159565, 1159567, 1159569, 1159571, 1159573, 1159575, 1159577, 1159579, 1159581, 1159583, 1159585, 1159587, 1159589, 1159591, 1159593, 1159595, 1159597, 1159599, 1159601, 1159603, 1159605, 1159607, 1159609, 1159611, 1159613, 1159615, 1159617, 1159619, 1159621, 1159623, 1159625, 1159627, 1159629, 1159631, 1159633, 1159635, 1159637, 1159639, 1159641, 1159643, 1159645, 1159647, 1159649, 1159651, 1159653, 1159655, 1159657, 1159659, 1159661, 1159663, 1159665, 1159667, 1159669, 1159671, 1159673, 1159675, 1159677, 1159679, 1159681, 1159683, 1159685, 1159687, 1159689, 1159691, 1159693, 1159695, 1159697, 1159699, 1159701, 1159703, 1159705, 1159707, 1159709, 1159711, 1159713, 1159715, 1159717, 1159719, 1159721, 1159723, 1159725, 1159727, 1159729, 1159731, 1159733, 1159735, 1159737, 1159739, 1159741, 1159743, 1159745, 1160571, 1160573, 1160575, 1160577, 1160579, 1160581, 1160583, 1160585, 1160587, 1160589, 1160591, 1160593, 1160595, 1160597, 1160599, 1160601, 1160603, 1160605, 1160607, 1160609, 1160613, 1160615, 1160617, 1160619, 1160705, 1160707, 1160709, 1160711, 1160713, 1160715, 1160717, 1160719, 1160721, 1160723, 1160725, 1160727, 1160729, 1160731, 1160733, 1160735, 1160737, 1160739, 1160743, 1160745, 1160747, 1160749, 1160751, 1160753, 1078251, 1078253, 1078255, 1078257, 1078259, 1078261, 1078263, 1078265, 1078267, 1078269, 1078271, 1078273, 1078275, 1078277, 4452848, 1078279, 1078281, 1078283, 1078285, 1078287, 1078289, 1078291, 4453396, 1078293, 1078295, 1078297, 1078299, 1111069, 1111071, 1078305, 1078307, 1111077, 1111079, 1111081, 1078315, 1111085, 1111087, 1111089, 1111091, 1111093, 1111095, 1111097, 1111099, 1111101, 1111107, 1111111, 1111113, 1111115, 1111117, 1111119, 1111121, 1111123, 1111125, 1078365, 1078367, 1078369, 1078373, 1078375, 1078379, 1078381, 1078383, 1078385, 1111155, 1078389, 1078391, 1078393, 1078397, 1078399, 1078401, 1078403, 1078405, 4453516, 1111261, 1111263, 1111265, 1111267, 1111269, 1111271, 1111273, 1078507, 1111277, 1078511, 1111281, 1111283, 1111285, 1111287, 1111289, 1111291, 1078525, 1111295, 4452850, 1111297, 1111299, 1111301, 1111303, 1111305, 1111307, 1111309, 1111311, 1111313, 1111315, 1078549, 1078551, 1078553, 1078555, 1078557, 1078559, 1078561, 1111331, 1111333, 1078573, 1078575, 1078577, 1078579, 1078581, 1078583, 1078585, 1078587, 1111357, 1078591, 4453696, 1078593, 1078595, 1111365, 1111367, 1111369, 1111371, 1111373, 1111375, 1111377, 1078611, 1078613, 1078615, 1078617, 1078619, 1111389, 1111391, 1078625, 1078627, 1078629, 1078631, 1078633, 1078635, 1078637, 1111407, 1078641, 1078643, 1078645, 1078647, 1078649, 1078651, 1078653, 1078655, 1078657, 1078659, 1078661, 1078663, 1078665, 1078667, 1078669, 1078671, 1078673, 1078675, 1078677, 1078679, 1078681, 1078683, 1078685, 1078687, 1078689, 1078691, 1078693, 1078695, 1078697, 1078699, 1078701, 1078703, 1078705, 1078707, 1078709, 1078711, 1078713, 1078715, 1078717, 1078719, 1078721, 1078723, 1078725, 1078727, 1078729, 1078731, 1078733, 1078735, 1078737, 1078739, 1078741, 1078743, 1078745, 1078747, 1078749, 1078751, 1078753, 1111525, 1078761, 1078763, 1078765, 1078767, 1078769, 1078771, 1078773, 1111543, 1078777, 1078779, 1078781, 1078783, 1078785, 1078787, 1078789, 1078791, 1078793, 1078795, 1078797, 1078799, 1078801, 1078803, 1078805, 1078807, 1078809, 1078811, 1078813, 1477505, 1164637, 1164639, 1164641, 1164643, 1164645, 1164647, 1164649, 1164651, 1164653, 1164655, 1164657, 1164659, 1164661, 1164663, 1164665, 1164667, 1164669, 1164671, 1164673, 1164675, 1164677, 1164679, 1164681, 1164683, 1164685, 1164687, 1164689, 1164691, 1164693, 1164695, 1164697, 1164699, 1164701, 1164703, 1164705, 1164707, 1164709, 1164711, 1164713, 1164715, 1164717, 1164719, 1164721, 1164723, 1164725, 1164727, 1164729, 1164731, 1164733, 1164735, 1164737, 1164739, 1164741, 1164743, 1164745, 1164747, 1164749, 1164751, 1164753, 1164755, 1164757, 1164759, 1164761, 1164763, 1164765, 1164767, 1164769, 1164771, 1164773, 1164775, 1164777, 1164779, 1164781, 1164783, 1164785, 1164787, 1164789, 1164791, 1164793, 1164795, 1164797, 1164799, 1164801, 1164803, 1164805, 1164807, 1164809, 1164811, 1164813, 1164815, 1164817, 1164819, 1164821, 1164823, 1164825, 1164827, 1164829, 1164831, 1164833, 1164835, 1164837, 1164839, 1164841, 1164843, 1164845, 1164847, 1164849, 1164851, 1164853, 1164855, 1164857, 1164859, 1164861, 1164863, 1164865, 1164867, 1164869, 1164871, 1164873, 1164875, 1164877, 1164879, 1164881, 1164883, 1164885, 1164887, 1164889, 1164891, 1164893, 1164895, 1164897, 1164899, 1164901, 1164903, 1164905, 1164907, 1164909, 1164911, 1164913, 1164915, 1164917, 1164919, 1164921, 1164923, 1164925, 1164927, 1164929, 1164931, 1164933, 1164935, 1164937, 1164939, 1164941, 1164943, 1164945, 1164947, 1164949, 1164951, 1164953, 1164955, 1164957, 1164959, 1164961, 1164963, 1164965, 1164967, 1164969, 1164971, 1164973, 1164975, 1164977, 1164979, 1164981, 1164983, 1164985, 1164987, 1164989, 1164991, 1164993, 1164995, 1164997, 1164999, 1165001, 1165003, 1165005, 1165007, 1165009, 1165011, 1165013, 1165015, 1165017, 1165019, 1165021, 1165023, 1165025, 1165027, 1165029, 1165031, 1165033, 1165035, 1165037, 1165039, 1165041, 1165043, 1165045, 1165047, 1165049, 1165051, 1165053, 1165055, 1165057, 1165059, 1165061, 1165063, 1165065, 1165067, 1165069, 1165071, 1165073, 1165075, 1165077, 1165079, 1165081, 1165083, 1165085, 1165087, 1165089, 1165091, 1165093, 1165095, 1165097, 1165099, 1165101, 1165103, 1165105, 1165107, 1165109, 1165111, 1165113, 1165115, 1165117, 1165119, 1165121, 1165123, 1165125, 1165129, 1165131, 1165133, 1165135, 1165137, 1165139, 1165223, 1165225, 1165227, 1165229, 1165231, 1165233, 1165235, 1165237, 1165239, 1165241, 1165243, 1165245, 1165247, 1165249, 1165251, 1165253, 1165255, 1165257, 1165259, 1165261, 1165263, 1165265, 1165267, 1165269, 1165271, 1165273, 1165275, 1165277, 1165279, 1165281, 1165283, 1165285, 1165287, 1165289, 1165291, 1165293, 1165295, 1165297, 1165299, 1165301, 1165303, 1165305, 1165307, 1165309, 1165311, 1165313, 1165315, 1165317, 1165319, 1165321, 1165323, 1165325, 1165327, 1165329, 1165331, 1165333, 1165335, 1165337, 1165339, 1165341, 1165343, 1165345, 1165347, 1165349, 1165351, 1165353, 1165355, 1165357, 1165359, 1165361, 1165363, 1165365, 1165367, 1165369, 1165371, 1165373, 1165375, 1165377, 1165379, 1165381, 1165383, 1165385, 1165387, 1165389, 1165391, 1165393, 1165395, 1165397, 1165399, 1165401, 1165403, 1165405, 1165407, 1165409, 1165411, 1165413, 1165415, 1165417, 1165419, 1165421, 1165423, 1165425, 1165427, 1150017, 1166329, 1166331, 1166333, 1166335, 1166337, 1166339, 1166341, 1166343, 1166345, 1166347, 1166349, 1166351, 1166353, 1166355, 1166357, 1166359, 1166361, 1166363, 1166365, 1166367, 1166369, 1166371, 1166373, 1166375, 1166377, 1166379, 1166381, 1166383, 1166385, 1166387, 1166389, 1166391, 1166393, 1166395, 1166397, 1166399, 1166401, 1166403, 1166405, 1166407, 1166409, 1166411, 1166413, 1166415, 1166417, 1166419, 1166421, 1166423, 1166425, 1166427, 1166429, 1166431, 1166433, 1166435, 1166437, 1166439, 1166441, 1166443, 1166445, 1166447, 1166449, 1166451, 1166453, 1166455, 1166457, 1166459, 1166461, 1166463, 1166465, 1166467, 1166469, 1166471, 1166473, 1166475, 1166477, 1166479, 1166481, 1166483, 1166485, 1166487, 1166489, 1166491, 1166493, 1166495, 1166497, 1166499, 1166501, 1166503, 1166505, 1166507, 1166509, 1166511, 1166513, 1166515, 1166517, 1166519, 1166521, 1166523, 1166525, 1166583, 1166585, 1166587, 1166589, 1166591, 1166593, 1166595, 1166597, 1166599, 1166601, 1166603, 1166605, 1166607, 1166609, 1166611, 1166613, 1166615, 1166617, 1166619, 1166621, 1166623, 1166625, 1166627, 1166629, 1166631, 1167449, 4452794, 1112437, 1112447, 1112453, 1079687, 1112457, 1112459, 1112461, 24599208, 1113893, 24599575, 24599577, 1079853, 1143208, 1143210, 1143212, 1143214, 1143218, 1143220, 1143222, 1143224, 1143230, 1079973, 24600591, 1143240, 1080033, 1080093, 1143276, 1143278, 1143286, 1118589, 1081119, 1081121, 1151397, 1151399, 1151401, 1151403, 1151405, 1081123, 4450864, 4450866, 4450868, 4450870, 4450872, 4450874, 4450878, 4450880, 4450882, 4450884, 4450886, 4450888, 4450890, 4450892, 4450894, 4450896, 4450898, 4450900, 4450902, 4450950, 1081127, 1081129, 1081131, 1141518, 1141520, 1081133, 1081135, 1081137, 4451430, 4451432, 4451434, 4451436, 4451440, 4451442, 4451444, 4451448, 4451450, 4451452, 4451454, 4451456, 4451458, 4451460, 4451462, 4451464, 4451466, 4451468, 4451470, 4451472, 4451474, 4451476, 4451478, 4451480, 4451482, 4451484, 4451486, 4451488, 4451490, 4451492, 4451494, 4451496, 4451498, 4451500, 4451502, 4451504, 4451506, 4451508, 4451510, 4451512, 4451514, 4451516, 4451518, 4451520, 4451522, 4451524, 4451526, 4451528, 4451530, 4451532, 4451534, 4451536, 4451538, 4451540, 4451542, 4451544, 4451546, 4451548, 4451550, 4451552, 4451554, 4451556, 4451558, 4451560, 4451562, 4451564, 4451566, 4451568, 4451570, 4451572, 4451574, 4451576, 4451578, 4451580, 4451582, 4451584, 4451586, 4451588, 4451590, 4451592, 4451594, 4451596, 4451598, 4451600, 4451602, 4451604, 4451606, 4451608, 1109273, 4451610, 1109275, 4451612, 1109277, 4451614, 1109279, 4451616, 1109281, 4451618, 1109283, 4451620, 1109285, 4451622, 4451624, 4451626, 4451628, 4451630, 4451632, 4451634, 4451636, 4451638, 4451640, 4451642, 4451644, 4451646, 4451648, 4451650, 4451652, 4451654, 4451656, 4451658, 4451660, 4451662, 4451664, 4451666, 4451668, 4451670, 4451672, 4451674, 4451676, 4451678, 4451680, 4451682, 4451684, 4451686, 4451688, 4451690, 4451692, 4451694, 4451696, 4451698, 4451700, 4451702, 4451704, 4451706, 4451708, 4451710, 4451712, 4451714, 4451716, 4451718, 4451720, 4451722, 4451724, 4451726, 4451728, 4451730, 4451732, 4451734, 4451736, 4451738, 4451740, 4451742, 4451744, 4451746, 4451748, 4451750, 4451752, 4451754, 4451756, 4451758, 4451760, 4451762, 4451764, 4451766, 4451768, 4451770, 4451772, 4451774, 4451776, 4451778, 4451780, 4451782, 4451784, 4451786, 4451788, 4451790, 4451792, 4451794, 4451796, 4451798, 4451800, 4451802, 4451804, 4451806, 4451808, 4451810, 4451812, 4451814, 4451816, 1076713, 4451818, 1076715, 4451820, 1076717, 4451822, 4451824, 4451826, 4451828, 4451830, 4451832, 4451834, 4451836, 4451838, 4451840, 4451842, 4451844, 4451846, 4451848, 4451850, 4451852, 4451854, 4451856, 4451858, 4451860, 4451862, 4451864, 4451866, 4451868, 4451870, 4451872, 4451874, 4451876, 4451878, 4451880, 4451882, 4451884, 4451886, 4451888, 4451890, 4451892, 4451894, 4451896, 4451898, 4451900, 4451902, 4451904, 4451906, 4451908, 4451910, 4451912, 4451914, 4451916, 4451918, 4451920, 4451922, 1109587, 4451924, 1109589, 4451926, 1109591, 4451928, 1109593, 4451930, 1109595, 4451932, 1109597, 4451934, 1109599, 4451936, 1109601, 4451938, 1109603, 4451940, 1109605, 4451942, 1109607, 4451944, 1109609, 4451946, 1109611, 4451948, 1109613, 4451950, 1109615, 4451952, 4451954, 1109619, 4451956, 4451958, 1109623, 4451960, 1109625, 4451962, 1109627, 4451964, 1109629, 4451966, 1109631, 4451968, 1109633, 4451970, 1109635, 4451972, 1109637, 4451974, 1109639, 4451976, 1109641, 4451978, 1109643, 4451980, 1109645, 4451982, 1109647, 4451984, 1109649, 4451986, 1109651, 4451988, 1109653, 4451990, 1109655, 4451992, 1109657, 4451994, 1109659, 4451996, 1109661, 4451998, 1109663, 4452000, 1109665, 4452002, 1109667, 4452004, 1109669, 4452006, 1109671, 4452008, 1109673, 4452010, 1109675, 4452012, 1109677, 4452014, 1109679, 4452016, 1109681, 4452018, 1109683, 4452020, 1109685, 4452022, 1109687, 4452024, 1109689, 4452026, 1109691, 1109693, 1109695, 1109697, 1109699, 1109701, 1109703, 4452066, 4452068, 4452070, 4452072, 4452074, 4452076, 4452078, 4452080, 4452082, 4452084, 4452086, 4452088, 1109753, 4452090, 1109755, 4452092, 1109757, 4452094, 1109759, 4452096, 4452098, 4452100, 4452102, 4452104, 4452106, 4452108, 4452110, 4452112, 4452114, 4452116, 4452118, 4452120, 4452122, 4452124, 4452126, 4452128, 4452130, 4452132, 4452134, 4452136, 4452138, 4452140, 4452142, 4452144, 4452146, 4452148, 4452150, 4452152, 4452154, 4452156, 4452158, 4452160, 4452162, 4452164, 4452166, 4452168, 4452170, 4452172, 4452174, 4452176, 4452178, 4452180, 4452182, 4452184, 4452186, 4452188, 4452190, 4452192, 4452194, 4452196, 4452198, 4452200, 4452202, 4452204, 4452206, 4452208, 4452210, 4452212, 4452214, 4452216, 4452218, 4452220, 4452222, 4452224, 4452226, 4452228, 4452230, 4452232, 4452234, 4452236, 4452238, 4452240, 4452242, 4452244, 4452246, 4452248, 4452250, 4452252, 4452254, 4452256, 4452258, 4452260, 4452262, 4452264, 4452266, 1142700, 1142702, 1142704, 4452274, 1142708, 4452278, 1142712, 1142714, 4452284, 4452286, 4452288, 4452290, 1142724, 1142726, 1142728, 1142730, 1142732, 1142734, 1142736, 1142738, 1142740, 1142742, 1142744, 1142746, 1142748, 1142750, 1142752, 1142754, 4452324, 1142758, 1142760, 1142762, 1142764, 4452334, 1142768, 4452338, 1142772, 1142774, 1142776, 1113907, 1142778, 1142780, 1142782, 1142784, 1142786, 1142788, 1142790, 1142792, 1142794, 1142796, 1142798, 1142800, 1142802, 1142804, 1142806, 1142808, 1142810, 1142812, 1142814, 1142816, 1142818, 1142820, 1142822, 1142824, 1142826, 1142828, 1142830, 1142832, 1142834, 1142836, 1142838, 1142840, 1142842, 1142844, 1142848, 1142850, 1142852, 1142854, 1142856, 1142858, 1142860, 1142862, 1142864, 1142866, 1142868, 1142870, 1142872, 1142874, 1142876, 1142878, 1142880, 1142882, 1142884, 1142886, 1142890, 1142894, 1142896, 1142898, 1142900, 1142902, 1142904, 1142906, 1142908, 1142910, 1142912, 1142914, 1142916, 1142918, 1142920, 1142922, 1110163, 1142932, 1142934, 1142936, 1142938, 1142940, 1110173, 1142942, 1110175, 1142944, 1110177, 1142946, 1110179, 1142948, 1110181, 1142950, 1110183, 1142952, 1110185, 1142954, 1110187, 1142956, 1110189, 1142958, 1110191, 1142960, 1110193, 1142962, 1110195, 1142964, 1142968, 1142970, 1142972, 1142974, 1142976, 1142978, 1142980, 1142982, 1142984, 1142986, 1142988, 1110221, 1142992, 1142994, 1142996, 1142998, 1143000, 1143002, 1143006, 1143008, 1143010, 1143012, 1143014, 1143016, 1143022, 1143026, 1143028, 1143030, 1143032, 1143034, 1143036, 1143044, 4452774, 1143054, 1143056, 1143058, 1143060, 1143062, 1143064, 1143066, 1143068, 1143078, 1143080, 1143088, 1143090, 1143096, 4452666, 4452668, 4452670, 4452672, 4452674, 4452676, 4452678, 4452680, 4452682, 4452684, 4452686, 4452688, 4452690, 4452692, 4452694, 1077591, 4452696, 1077593, 4452698, 1077595, 4452700, 1077597, 4452702, 1077599, 4452704, 1077601, 4452706, 1077603, 1143140, 1077605, 4452710, 1077607, 4452712, 1077609, 4452714, 1077611, 4452716, 1077613, 4452718, 1077615, 4452720, 1077617, 4452722, 1077619, 4452724, 1077621, 4452726, 1077623, 4452728, 1077625, 4452730, 1077627, 4452732, 1077629, 4452734, 1077631, 1077633, 4452738, 1077635, 4452740, 1077637, 4452742, 1077639, 4452744, 1077641, 4452746, 1077643, 4452748, 1077645, 4452750, 1077647, 4452752, 1077649, 4452754, 1077651, 4452756, 1077653, 4452758, 1077655, 4452760, 1077657, 4452762, 1077659, 4452764, 1077661, 4452766, 1077663, 4452768, 1077665, 4452770, 1077667, 4452772, 1143206, 1077671, 4452776, 4452778, 1077675, 4452780, 1077677, 4452782, 1077679, 1143216, 1077681, 4452786, 1077683, 4452788, 1077685, 4452790, 1077687, 4452792, 1077689, 1143226, 1077691, 4452796, 1077693, 4452798, 1077695, 1143232, 1077697, 4452802, 1077699, 1143236, 1077701, 4452806, 1077703, 4452808, 1077705, 1143242, 1077707, 4452812, 1077709, 4452814, 1077711, 4452816, 1077713, 4452818, 1077715, 4452820, 1077717, 4452822, 1077719, 4452824, 1077721, 1143258, 1077723, 1143260, 1077725, 4452830, 1077727, 1143264, 1077729, 4452834, 1077731, 1143268, 1077733, 1143270, 1077735, 1143272, 1077737, 1143274, 1077739, 4452844, 1077741, 4452846, 1143280, 1143282, 1143284, 4452854, 1143288, 1143290, 1143292, 4452862, 1143296, 1143298, 1143300, 1143302, 1143304, 1143306, 4452878, 4452880, 1143314, 1143316, 4452886, 4452888, 4452890, 4452892, 4452894, 4452896, 4452898, 4452900, 4452904, 4452906, 4452908, 4452910, 4452784, 4452916, 4452918, 4452920, 4452922, 4452924, 4452926, 4452928, 4452930, 4452932, 4452934, 4452936, 4452938, 4452940, 4452942, 4452944, 4452946, 4452948, 4452950, 4452952, 4452954, 4452956, 4452958, 4452960, 4452962, 1077859, 4452964, 1077861, 4452966, 1077863, 4452968, 1077865, 4452970, 1077867, 4452972, 1077869, 4452974, 1077871, 4452976, 4452978, 4452980, 4452982, 4452984, 4452986, 4452988, 4452990, 4452992, 4452994, 4452996, 4452998, 4453000, 4453002, 4453004, 4453006, 4453008, 4453010, 4453012, 4453014, 4453016, 4453018, 4453020, 4453022, 4453024, 4453026, 4453028, 4453030, 4453032, 1110697, 4453034, 1110699, 4453036, 1110701, 4453038, 1110703, 4453040, 1110705, 4453042, 1110707, 4453044, 1110709, 4453046, 1110711, 4453048, 1110713, 4453050, 1110715, 4453052, 1110717, 4453054, 1110719, 4453056, 1110721, 4453058, 1110723, 4453060, 1110725, 4453062, 1110727, 4453064, 1110729, 4453066, 1110731, 4453068, 1110733, 4453070, 1110735, 4453072, 1110737, 4453074, 1110739, 4453076, 1110741, 4453078, 1110743, 4453080, 1110745, 4453082, 1110747, 4453084, 1110749, 4453086, 1110751, 4453088, 1110753, 4453090, 1110755, 4453092, 1110757, 4453094, 1110759, 4453096, 1110761, 4453098, 1110763, 4453100, 1110765, 4453102, 1110767, 4453104, 1110769, 4453106, 1110771, 4453108, 1110773, 4453110, 1110775, 4453112, 1110777, 4453114, 1110779, 4453116, 1110781, 4453118, 1110783, 4453120, 1110785, 4453122, 1110787, 4453124, 1110789, 4453126, 1110791, 4453128, 1110793, 4453130, 1110795, 4453132, 1110797, 4453134, 1110799, 4453136, 1110801, 4453138, 1110803, 4453140, 1110805, 4453142, 1110807, 4453144, 1110809, 4453146, 1110811, 4453148, 1110813, 4453150, 1110815, 4453152, 1110817, 4453154, 1110819, 4453156, 1110821, 4453158, 1110823, 4453160, 1110825, 4453162, 1110827, 4453164, 1110829, 4453166, 1110831, 4453168, 1110833, 4453170, 1110835, 4453172, 1110837, 4453174, 1110839, 4453176, 1110841, 4453178, 1110843, 4453180, 1110845, 4453182, 1110847, 4453184, 1110849, 4453186, 1110851, 4453188, 1110853, 4453190, 1110855, 4453192, 1110857, 4453194, 1110859, 4453196, 1110861, 4453198, 1110863, 4453200, 1110865, 4453202, 1110867, 4453204, 1110869, 4453206, 1110871, 4453208, 1110873, 4453210, 1110875, 4453212, 1110877, 4453214, 1110879, 4453216, 1110881, 4453218, 1110883, 4453220, 1110885, 4453222, 1110887, 4453224, 1110889, 4453226, 1110891, 4453228, 1110893, 4453230, 1110895, 4453232, 1110897, 4453234, 1110899, 4453236, 1110901, 4453238, 1110903, 4453240, 1110905, 4453242, 1110907, 4453244, 1110909, 4453246, 1110911, 4453248, 1110913, 4453250, 1110915, 4453252, 1110917, 4453254, 1110919, 4453256, 1110921, 4453258, 1110923, 4453260, 1110925, 4453262, 1110927, 4453264, 1110929, 4453266, 1110931, 4453268, 1110933, 4453270, 1110935, 4453272, 1110937, 4453274, 1110939, 4453276, 1110941, 4453278, 1110943, 4453280, 1110945, 4453282, 1110947, 4453284, 1110949, 4453286, 1110951, 4453288, 1110953, 4453290, 1110955, 4453292, 1110957, 4453294, 1110959, 4453296, 1110961, 4453298, 1110963, 4453300, 1110965, 4453302, 1110967, 4453304, 1110969, 4453306, 1110971, 4453308, 1110973, 4453310, 1110975, 4453312, 1110977, 4453314, 1110979, 4453316, 1110981, 4453318, 1110983, 4453320, 1110985, 4453322, 1110987, 4453324, 1110989, 4453326, 1110991, 4453328, 1110993, 4453330, 1110995, 4453332, 1110997, 4453334, 1110999, 4453336, 1111001, 4453338, 1111003, 4453340, 1111005, 4453342, 1111007, 4453344, 1111009, 4453346, 1111011, 4453348, 1111013, 4453350, 1111015, 4453352, 1111017, 4453354, 1111019, 4453356, 1111021, 4453358, 1111023, 4453360, 1111025, 4453362, 1111027, 4453364, 1111029, 4453366, 1111031, 4453368, 1111033, 4453370, 1111035, 4453372, 1111037, 4453374, 1111039, 4453376, 1111041, 4453378, 1111043, 4453380, 1111045, 4453382, 1111047, 4453384, 1111049, 4453386, 1111051, 4453388, 1111053, 4453390, 1111055, 4453392, 1111057, 4453394, 1111059, 4452800, 1111061, 4453398, 1111063, 4453400, 1111065, 4453402, 1111067, 4453404, 1078301, 4453406, 1078303, 4453408, 1111073, 4453410, 1111075, 4453412, 1078309, 4453414, 1078311, 4453416, 1078313, 4453418, 1111083, 4453420, 1078317, 4453422, 1078319, 4453424, 1078321, 4453426, 1078323, 4453428, 1078325, 4453430, 1078327, 4453432, 1078329, 4453434, 1078331, 4453436, 1078333, 4453438, 1078335, 4453440, 1078337, 4453442, 1078339, 4453444, 1078341, 4453446, 1078343, 4453448, 1078345, 4453450, 1078347, 4453452, 1078349, 4453454, 1078351, 4453456, 1078353, 4453458, 1078355, 4453460, 1078357, 4453462, 1078359, 4453464, 1078361, 4453466, 1078363, 1111133, 4453470, 1111135, 4453472, 1111137, 4453474, 1078371, 4453476, 1111141, 4453478, 1111143, 4453480, 1078377, 4453482, 1111147, 4453484, 1111149, 4453486, 1111151, 4453488, 1111153, 4453490, 1078387, 4453492, 1111157, 4453494, 1111159, 4453496, 1111161, 4453498, 1111163, 4453500, 1111165, 4453502, 1111167, 4453504, 1111169, 4453506, 1111171, 4453508, 1111173, 4453510, 1078407, 4453512, 1078409, 4453514, 1078411, 4452804, 1078413, 4453518, 1078415, 4453520, 1078417, 4453522, 1078419, 4453524, 1078421, 4453526, 1078423, 4453528, 1078425, 4453530, 1078427, 4453532, 1078429, 4453534, 1078431, 4453536, 1078433, 4453538, 1078435, 4453540, 1078437, 4453542, 1078439, 4453544, 1078441, 4453546, 1078443, 4453548, 1078445, 4453550, 1078447, 4453552, 1078449, 4453554, 1078451, 4453556, 1078453, 4453558, 1078455, 4453560, 1078457, 4453562, 1078459, 4453564, 1078461, 4453566, 1078463, 4453568, 1078465, 4453570, 1078467, 4453572, 1078469, 1078471, 1078473, 4453578, 1078475, 4453580, 1078477, 4453582, 1078479, 4453584, 4453586, 1078483, 4453588, 1078485, 4453590, 1078487, 4453592, 1078489, 4453594, 1078491, 4453596, 1078493, 4453598, 1078495, 4453600, 1078497, 1078499, 4453604, 1078501, 4453606, 1078503, 4453608, 1078505, 4453610, 1111275, 4453612, 1078509, 4453614, 1111279, 4453616, 1078513, 4453618, 1078515, 4453620, 1078517, 4453622, 1078519, 4453624, 1078521, 4453626, 1078523, 4453628, 1111293, 4453630, 1078527, 4453632, 1078529, 4453634, 1078531, 4453636, 1078533, 4453638, 1078535, 4453640, 1078537, 4453642, 1078539, 4453644, 1078541, 4453646, 1078543, 4453648, 1078545, 4453650, 1078547, 4453652, 1111317, 4453654, 1111319, 4453656, 1111321, 4453658, 1111323, 4453660, 1111325, 4453662, 1111327, 4453664, 1111329, 4453666, 1078563, 4453668, 1078565, 4453670, 1078567, 4453672, 1078569, 4453674, 1078571, 4453676, 1111341, 4453678, 1111343, 4453680, 1111345, 4453682, 1111347, 4453684, 1111349, 4453686, 1111351, 4453688, 1111353, 4453690, 1111355, 4453692, 1078589, 4453694, 1111359, 4452810, 1111361, 4453698, 1111363, 4453700, 1078597, 4453702, 1078599, 4453704, 1078601, 4453706, 1078603, 4453708, 1078605, 4453710, 1078607, 4453712, 1078609, 4453714, 1111379, 4453716, 1111381, 4453718, 1111383, 4453720, 1111385, 4453722, 1111387, 4453724, 1078621, 4453726, 1078623, 4453728, 1111393, 4453730, 1111395, 4453732, 1111397, 4453734, 1111399, 4453736, 1111401, 4453738, 1111403, 4453740, 1111405, 4453742, 1078639, 4453744, 1111409, 4453746, 1111411, 4453748, 1111413, 4453750, 1111415, 4453752, 1111417, 4453754, 1111419, 4453756, 1111421, 4453758, 1111423, 4453760, 1111425, 4453762, 1111427, 4453764, 1111429, 4453766, 1111431, 4453768, 1111433, 4453770, 1111435, 4453772, 1111437, 4453774, 1111439, 4453776, 1111441, 4453778, 1111443, 4453780, 1111445, 4453782, 1111447, 4453784, 1111449, 4453786, 1111451, 4453788, 1111453, 4453790, 1111455, 4453792, 1111457, 4453794, 1111459, 4453796, 1111461, 4453798, 1111463, 4453800, 1111465, 4453802, 1111467, 4453804, 1111469, 4453806, 1111471, 4453808, 1111473, 4453810, 1111475, 4453812, 1111477, 4453814, 1111479, 4453816, 1111481, 4453818, 1111483, 4453820, 1111485, 4453822, 1111487, 4453824, 1111489, 4453826, 1111491, 4453828, 1111493, 4453830, 1111495, 4453832, 1111497, 4453834, 1111499, 4453836, 1111501, 4453838, 1111503, 4453840, 1111505, 4453842, 1111507, 4453844, 1111509, 4453846, 1111511, 4453848, 1111513, 4453850, 1111515, 4453852, 1111517, 4453854, 1111519, 4453856, 1111521, 4453858, 1111523, 4453860, 1078757, 4453862, 1111527, 4453864, 1111529, 4453866, 1111531, 1111533, 1111535, 1111537, 1111539, 1111541, 1078775, 1111545, 1111547, 1111549, 1111551, 1111553, 1111555, 1111557, 1111559, 1111561, 1111563, 1111565, 1111567, 1111569, 1111571, 1111573, 1111575, 1111577, 1111579, 1111581, 1078815, 1078817, 1078819, 1078821, 1078823, 1078825, 1078827, 1078829, 1078831, 1078833, 1078835, 1078837, 1078839, 1078841, 1078843, 1078845, 1078847, 1078849, 1078851, 1078853, 1078855, 1078857, 1078859, 1078861, 1078863, 1078865, 1078867, 1078869, 1078871, 1078873, 1078875, 1078877, 1078879, 4453984, 1078881, 4453986, 1078883, 4453988, 1078885, 4453990, 1078887, 4453992, 1078889, 4453994, 1078891, 4453996, 1078893, 4453998, 1078895, 4454000, 1078897, 4454002, 1078899, 4454004, 1078901, 4454006, 1078903, 4454008, 1078905, 4454010, 1078907, 4454012, 1078909, 4454014, 1078911, 4454016, 1078913, 4454018, 1078915, 4454020, 1078917, 4454022, 1078919, 4454024, 1078921, 4454026, 1078923, 1078925, 4454030, 1078927, 4454032, 1078929, 4454034, 1078931, 4454036, 1078933, 4454038, 1078935, 4454040, 1078937, 4454042, 1078939, 4454044, 1078941, 4454046, 1078943, 4454048, 1078945, 4454050, 1078947, 4454052, 1078949, 4454054, 1078951, 4454056, 1078953, 4454058, 1078955, 4454060, 4454062, 4454064, 4454066, 4454068, 4454070, 4454072, 4454074, 4454076, 4454078, 4454080, 4454082, 4454084, 4454086, 4454088, 4454090, 4454092, 4454094, 4454096, 4454098, 4454100, 4454102, 4454104, 4454106, 4454108, 1079005, 4454110, 1079007, 4454112, 1079009, 4454114, 1079011, 4454116, 1079013, 4454118, 1079015, 4454120, 1079017, 4454122, 1079019, 4454124, 1079021, 4454126, 1079023, 4454128, 1079025, 4454130, 1079027, 4454132, 1079029, 4454134, 1079031, 4454136, 1079033, 4454138, 1079035, 4454140, 1079037, 4454142, 1079039, 4454144, 1079041, 4454146, 1079043, 4454148, 1079045, 4454150, 1079047, 4454152, 1079049, 4454154, 1079051, 4454156, 1079053, 4454158, 1079055, 4454160, 1079057, 4454162, 4454164, 1079061, 4454166, 1079063, 4454168, 1079065, 4454170, 1079067, 4454172, 1079069, 4454174, 1079071, 4454176, 1079073, 4454178, 1079075, 4454180, 1079077, 4454182, 1079079, 4454184, 1079081, 4454186, 1079083, 4454188, 1079085, 4454190, 4454192, 1079089, 4454194, 1079091, 4454196, 1079093, 4454198, 4454200, 4454202, 4454204, 4454206, 4454208, 4454210, 4454212, 4454214, 4454216, 4454218, 4454220, 4454222, 4454224, 4454226, 4454228, 4454230, 4454232, 4454234, 4454236, 4454238, 4454240, 4454242, 4454244, 4454246, 4454248, 4454250, 4454252, 4454254, 4454256, 4454258, 4454260, 4454262, 4454264, 4454266, 4454268, 4454270, 4454272, 4454274, 4454276, 4454278, 4454280, 4454282, 4454284, 4454286, 4454288, 4454290, 4454292, 4454294, 4454296, 4454298, 4454300, 4454302, 4454304, 4454306, 4454308, 4454310, 4454312, 4454314, 4454316, 4454318, 1080989, 4454320, 4454322, 4454324, 4454326, 4454328, 4454330, 1080991, 4454332, 4454334, 4454336, 4454338, 4454340, 1080993, 4454344, 4454346, 4454348, 4454350, 4454352, 4454354, 1080995, 4454356, 4452832, 4454358, 4454360, 4454362, 1080997, 1080999, 1081001, 4454400, 4454402, 1081003, 4454404, 4454406, 4454408, 4454410, 4454412, 4454414, 1081005, 4454416, 4454418, 4454420, 4454422, 4454426, 1081007, 4454428, 4454430, 4454432, 4454434, 4454436, 4454438, 1081009, 4454440, 4454442, 4454444, 4454446, 4454448, 4454450, 1081011, 4454452, 4454454, 4454456, 4454458, 4454460, 4454462, 1081013, 4454464, 4454466, 4454468, 4454470, 4454472, 4454474, 1081015, 4454476, 4452836, 4454478, 4454480, 1081017, 1081019, 1081021, 1081023, 4454532, 1081025, 4452838, 1081027, 1081029, 1081031, 4452840, 1112277, 1112279, 1112283, 4452842, 1112323, 1112325, 1112327, 1112333, 1079585, 1079587, 1079589, 1079591, 1079593, 1079595, 1079597, 1079599, 1079601, 1079603, 1079605, 1079607, 1079609, 1079611, 1079613, 1079615, 1079617, 1079619, 1079621, 1079623, 1079625, 1079627, 1079629, 1079631, 1079633, 1079635, 1079637, 1079639, 1079641, 1079643, 1079645, 1079647, 1079649, 1079651, 1079653, 1079655, 1079657, 1079659, 1079661, 1079663, 1079665, 1079667, 1079669, 1079671, 1079673, 1079675, 1079677, 1079679, 1079681, 1079683, 1079685, 1112455, 1079689, 1079691, 1079693, 1079695, 1079697, 1079699, 1079701, 1079703, 1079705, 1079707, 1079709, 1079711, 1079713, 1079715, 1079717, 1079719, 1079721, 1079723, 1079725, 1079727, 1079729, 1079731, 1079733, 1079735, 1079737, 1079739, 1079741, 1079743, 1079745, 1079747, 1079749, 1079751, 1079753, 1079755, 1079757, 1079759, 1079761, 1079763, 1079765, 1079767, 1079769, 1079771, 1079773, 1079775, 1079777, 1079779, 1079781, 1079783, 1079785, 1079787, 1079789, 1079791, 1079793, 1079795, 1079797, 1079799, 1079801, 1079803, 1079805, 1079807, 1079809, 1079811, 1079813, 1079815, 1079817, 1079819, 1079821, 1079823, 1079825, 1079827, 1079829, 1079831, 1079833, 1079835, 1079837, 1079839, 1079841, 1079843, 1079845, 1079847, 1079849, 1079851, 4452852, 1079855, 1079857, 1079859, 1079861, 1079863, 1079865, 1079867, 1079869, 1079871, 1079873, 1079875, 1079877, 1079879, 1079881, 1079883, 1079885, 1079887, 1079889, 1079891, 1145428, 1079893, 1079895, 1079897, 1079899, 1079901, 1079903, 1079905, 1079907, 1079909, 1079911, 1079913, 1079915, 1079917, 1079919, 1079921, 1079923, 1079925, 1079927, 1079929, 1079931, 1079933, 1079935, 1079937, 1079939, 1079941, 1079943, 1079945, 1079947, 1079949, 1079951, 1079953, 1079955, 1079957, 1079959, 1079961, 1079963, 1079965, 1079967, 1079969, 1079971, 4452856, 1079975, 1079977, 1079979, 1079981, 1079983, 1079985, 1079987, 1079989, 1079991, 1079993, 1079995, 1079997, 1079999, 1080001, 1080003, 1080005, 1080007, 1080009, 1080011, 1080013, 1080015, 1080017, 1080019, 1080021, 1080023, 1080025, 1080027, 1080029, 1080031, 4452858, 1080035, 1080037, 1080039, 1080041, 1080043, 1080045, 1080047, 1080049, 1080051, 1080053, 1080055, 1080057, 1080059, 1080061, 1080063, 1080065, 1080067, 1080069, 1080071, 1080073, 1080075, 1080077, 1080079, 1080081, 1080083, 1080085, 1080087, 1080089, 1080091, 4452860, 1080095, 1080097, 1080099, 1080101, 1080103, 1080105, 1080107, 1080109, 1080111, 1080115, 1080117, 1080119, 1080121, 1080123, 1080125, 1080127, 1080129, 1080131, 1080133, 1080135, 1080137, 1080139, 1080141, 1080143, 1080145, 1080147, 1080149, 1080151, 1080153, 1080155, 1080157, 1080159, 1080161, 1080163, 1080165, 1080167, 1080169, 1080171, 1080173, 1080175, 1080177, 1080179, 1080181, 1080183, 1080185, 1080187, 1080189, 1080191, 1080193, 1080195, 1080197, 1080201, 1080203, 1080255, 1080257, 1080259, 1080261, 1080263, 1080265, 1080267, 1080269, 1080271, 1080273, 1080275, 1080277, 1080279, 1080281, 1080283, 1080285, 1080287, 1080289, 1080291, 1080293, 1080295, 1080297, 1080299, 1080301, 1080805, 1080807, 1080809, 1080811, 1080813, 1080815, 1080817, 1080819, 1080821, 1080823, 1080825, 1080829, 1080831, 1080833, 1080837, 1080839, 1080841, 1080843, 1080845, 1080847, 1080849, 1080851, 1080853, 1080855, 1080857, 1080859, 1080861, 1080863, 1080865, 1080867, 1080869, 1080871, 1080873, 1080875, 1080877, 1080879, 1080881, 1080883, 1080885, 1080887, 1080889, 1080891, 1080893, 1080895, 1080897, 1080899, 1080901, 1080903, 1080905, 1080907, 1080909, 1080911, 1080913, 1080915, 1080917, 1080919, 1080921, 1080923, 1080925, 1080927, 1080929, 1080931, 1080933, 1080935, 1080937, 1080939, 1080941, 1080943, 1080945, 1080947, 1080949, 1080951, 1080953, 1080955, 1080957, 1080959, 1080961, 1080963, 1080965, 1080967, 1080969, 1080971, 1080973, 1080975, 1080977, 1080979, 1080981, 1080983, 1080985, 1080987, 1113757, 1113759, 1113761, 1113763, 1113765, 1113767, 1113769, 1113771, 1113773, 1113775, 1113777, 1113779, 1113781, 1113783, 1113785, 1113787, 1113789, 1113791, 1113793, 1113795, 1113797, 1113799, 1113801, 1113803, 1113805, 1113807, 1113809, 1113811, 1113813, 1113815, 1113817, 1113819, 1113821, 1113823, 1113825, 1113827, 1113829, 1113831, 1113833, 1113835, 1113837, 1113839, 1113841, 1113843, 1113845, 1113847, 1113849, 1113851, 1113853, 1113855, 1113857, 1113859, 1113861, 1113863, 1113865, 1113867, 1113869, 1113871, 1113873, 1113875, 1113877, 1113879, 1113881, 1113883, 1113885, 1113887, 1113889, 1113891, 1081125, 1113895, 1113897, 1113899, 1113901, 1113903, 1113905, 1081139, 1113909, 1113911, 1113913, 1113915, 1113917, 1113919, 1113921, 1113923, 1113925, 1113927, 1113929, 1113931, 1113933, 1113935, 1113937, 1113939, 1113941, 1113943, 1113945, 1113947, 1113949, 1113951, 1113955, 1113957, 1113961, 1113963, 1113965, 1113969, 1113971, 1113975, 1113977, 1113979, 1113983, 1113985, 1113987, 1113989, 1113991, 1113995, 1113999, 1114001, 1114003, 1114005, 1114007, 1114009, 1114011, 1114013, 1114017, 1114029, 1114103, 1114105, 1114111]

Find the indices that correspond with each of the COMIDs in the list above.  We'll end up with 5522 unique reaches.

In [ ]:
indices = np.where(channel_ds.feature_id.isin(reachids))[0]

Subset the complete `channel_rt` dataset using this list of indices. This will reduce the overall size of our dataset from 2,716,897 to 5522 reaches.

In [ ]:
channel_subset = channel_ds.isel(feature_id=indices)
channel_subset

Preview the values for one of the reaches.

In [ ]:
channel_subset.streamflow[:,1350].values

Plot data by specifying the index of the reach of interest, for example:

In [ ]:
comids = np.random.randint(5522, size=5)

fig, axes = plt.subplots(nrows=len(comids), figsize=(15,3*len(comids)))
axid = 0
for comid in comids:
    channel_subset.streamflow[:, comid].plot(ax=axes[axid])
    axid +=1
plt.tight_layout()

Save the subsetted `channel_rt` forecasts

In [ ]:
%%time

# save the data
data = channel_subset.load()

In [ ]:
data.to_netcdf(path='120200_channel_rt.nc')

## 3. Share Our Results

The best way to save your data is to put it back into HydroShare and is done using the `createHydroShareResource` function. The first step is to identify the files you want to save to a HydroShare.  The cell below lists all the files in the current working directory.

In [ ]:
from utilities import hydroshare
hs = hydroshare.hydroshare()

In [ ]:
# lets save this content as a new resource in HydroShare
title = 'Subsetted NWM Forecast - JupyterHub Demo'    
abstract = 'These are the results from the NWM forecast subsetting example that was performed in JupyterHub'
keywords = ['Demo', 'JupyterHub']  
rtype = 'compositeresource'          

files = ['120200_land_sat.nc', '120200_channel_rt.nc', 'Harvey-OpenDap-Subsetting.ipynb']

# create the new resource
resource_id = hs.createHydroShareResource(abstract, 
                                          title, 
                                          keywords=keywords, 
                                          resource_type=rtype, 
                                          content_files=files, 
                                          public=False)